# Social Media Monitoring

## Organization List


The following table contains data on the organizations and their associated social acounts

In [ ]:
import qgrid
from smm_wrapper import SMMOrganizations
qgrid.set_grid_option('maxVisibleRows', 5)

# Create an instance to the smm wrapper
smm = SMMOrganizations()

# Request the politicians from the API
df = smm.dv.get_organizations()
df = df[(df['category']=='media')]
# Show the result
qgrid.show_grid(df)

## Instructions

Please choose an `subcategory`, which indicates a type of media, from the table above and enter its name in the field `category` below:

In [ ]:
category = 'Newspaper'
aggregation = 'month' #aggregation can be by day ('day'), week ('week') or month('month')

In [ ]:
import pandas as pd

# Filter the accounts by party, and valid ones (the ones that contain fb_ids)
media_df = df[(df['subcategory'].str.contains(category)) & (df['fb_ids'].notnull())] 
media_df = media_df[(media_df['fb_ids'].str.len() == 1)]

# make sure that accounts are not repeated, e.g. Die Zeit (Online) and Die Zeit (Allgemeine) could
# share an account if facebook but not in twitter
fbids = set(fb_id for fb_ids in media_df['fb_ids'] for fb_id in fb_ids)

# query the Social Media Monitoring API
posts_by = pd.concat(smm.dv.posts_by(facebook_user_id=fb_id, from_date='2017-09-01', to_date='2018-12-31', aggregate_by=aggregation) 
               for fb_id in fbids)        
comments_by = pd.concat(smm.dv.comments_by(facebook_user_id=fb_id, from_date='2017-09-01', to_date='2018-12-31', aggregate_by=aggregation) 
               for fb_id in fbids) 

# aggregate the posts
total_posts_by = posts_by.groupby('date').agg(
    {
        'posts': sum,
        'replies': sum, 
        'shares': sum,
        'reactions': sum,
        'likes': sum
    })
# aggregate the comments
total_comments_by = comments_by.groupby('date').agg(
    {
        'comments': sum,
        'replies': sum,
        'likes': sum
    })

In [ ]:
media_df.head()

In [ ]:
mdf = media_df[media_df['fb_ids'].str.len() == 1].copy()
mdf['fbid'] = mdf['fb_ids'].str[0]

In [ ]:
mdf[mdf['fbid'].duplicated(False)]

## Plotting

In [ ]:
import plotly
from plotly import graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

#plot for fb posts
plotly.offline.iplot({
    "data": [go.Scatter(x=total_posts_by.index.tolist(), y=total_posts_by['posts'], name='Posts'), 
             go.Scatter(x=total_posts_by.index.tolist(), y=total_posts_by['replies'], name='Replies'),
            go.Scatter(x=total_posts_by.index.tolist(), y=total_posts_by['shares'], name='Shares'),
            go.Scatter(x=total_posts_by.index.tolist(), y=total_posts_by['reactions'], name='Reactions'),
            go.Scatter(x=total_posts_by.index.tolist(), y=total_posts_by['likes'], name='Likes')], 
    "layout": go.Layout(title='Facebook posts for {}'.format(category), xaxis={'title':''}, yaxis={'title':'N'})
})
#plot for fb comments
plotly.offline.iplot({
    "data": [go.Scatter(x=total_comments_by.index.tolist(), y=total_comments_by['comments'], name='Comments'), 
             go.Scatter(x=total_comments_by.index.tolist(), y=total_comments_by['replies'], name='Replies'),
            go.Scatter(x=total_comments_by.index.tolist(), y=total_comments_by['likes'], name='Likes')], 
    "layout": go.Layout(title='Facebook comments for {}'.format(category), xaxis={'title':''}, yaxis={'title':'N'})
})

In [ ]:
total_posts_by.loc['2017-12-01']